# Feature Engineering
Do feature engineering 
- <a href='#1'>1. tf idf </a> 
- <a href='#2'>2. matrix factorization </a> 
- <a href='#2'>2. w2v </a> 
- <a href='#3'>3. stats feat</a>

In [1]:
! pip install gensim -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
%load_ext autoreload
%autoreload 2
import sys
import os 
import gc
import functools

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import multiprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from tqdm import tqdm
import pyspark.sql.functions as F
from pyspark.sql import (
    SparkSession,
    Window,
    DataFrame as Spark_DataFrame
)

sys.path.append('../')
import conf
from utils import (
    check_columns, 
    check_nan_value,
    correct_column_type_by_value_range,
    LogManager,
    timer
)

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [2]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(11,4)})
spark = SparkSession.builder.appName('feature_engineering')\
.config('spark.debug.maxToStringFields',2000)\
.config('spark.num_executor','8')\
.config('spark.executor.memory','15g')\
.config('spark.driver.memory','15g')\
.config('spark.driver.maxResultSize', '10g')\
.config('spark.sql.execution.arrow.enabled',"true")\
.getOrCreate()
# .config('spark.num_executor','8')\
# .config('spark.executor.memory','2g')\
# .config('spark.driver.memory','10g')\
# .config('spark.master','local[4]')

In [3]:
# global variables
DEFAULT_MISSING_VALUE = 0
FONT = fm.FontProperties(fname = os.path.join(conf.LIB_DIR,'simsun.ttc'))
LogManager.created_filename = os.path.join(conf.LOG_DIR, 'feature_engineering.log')
logger = LogManager.get_logger(__name__)

In [4]:
# functions
def __dummy():
    pass

def generate_tf_idf_feat(df):
    pass

def generate_stats_feat():
    pass

### <a id='1'> 1. generate emb for sparse feat</a>

In [5]:
@timer(logger)
def _generate_emb_for_sparse_feat(
    df,
    sparse_feat,
    method,
    max_df=1.0,
    min_df=1,
    emb_dim=100,
    ngram_range = (1,1),
    window=5,
    min_count=5,
    workers=4,
    sg=0
):  
    sparse_feat_seq_df = df[['user_id',sparse_feat]]
    sparse_feat_seq_df[sparse_feat] = sparse_feat_seq_df[sparse_feat].astype(str)
    sparse_feat_seq_df = sparse_feat_seq_df.groupby(['user_id'])[sparse_feat].apply(list).reset_index()
    
    if method == 'tf_idf':
        logger.info('sparse_feat: %s , method: %s, emb_dim: %s, max_df: %s, min_df: %s, ngram_range: %s'%(
                                                                                  sparse_feat,
                                                                                  method,
                                                                                  emb_dim,
                                                                                  max_df,
                                                                                  min_df,
                                                                                  ngram_range,
    ))  
        sparse_feat_seq_df[sparse_feat] = sparse_feat_seq_df[sparse_feat].apply(lambda x:' '.join(x))
        tf = TfidfVectorizer(max_features=emb_dim, max_df=max_df, min_df=min_df, sublinear_tf=True,                                     ngram_range=ngram_range)
        sparse_feat_seq_arr = tf.fit_transform(sparse_feat_seq_df[sparse_feat].to_list()).toarray()
#         logger.info(sparse_feat_seq_arr.shape)
#         csr_df = pd.DataFrame.sparse.from_spmatrix(sparse_feat_seq_tf_csr)
#         csr_df.columns = ['tf_'+ str(col) for col in csr_df.columns]
#         sparse_feat_seq_df = pd.concat([sparse_feat_seq_df[['user_id']], csr_df])
    elif method == 'matrix_factorization':
        raise NotImplementedError
    elif method == 'w2v':
        logger.info('sparse_feat: %s , method: %s, emd_dim: %s, window: %s, min_count: %s, workers: %s, sg: %s'%(
                                                                                  sparse_feat,
                                                                                  method,
                                                                                  emb_dim,
                                                                                  window,
                                                                                  min_count,
                                                                                  workers,
                                                                                  sg
    ))
        sparse_feat_seq_list = sparse_feat_seq_df[sparse_feat].to_list()
        if os.path.exists(os.path.join(conf.DATA_DIR, '%s_w2v.bin'%sparse_feat)):
            w2v_model = KeyedVectors.load_word2vec_format(os.path.join(conf.DATA_DIR, '%s_w2v.bin'%sparse_feat), binary=True) 
        else:
            w2v_model = Word2Vec(
                                sparse_feat_seq_list,
                                window=window,
                                size=emb_dim,
                                min_count=min_count,
                                workers=workers,
                                sg=sg
                                )
            w2v_model.wv.save_word2vec_format(os.path.join(conf.DATA_DIR, '%s_w2v.bin'%sparse_feat), binary=True)
            
        sparse_feat_seq_arr = np.asarray([functools.reduce(lambda a,b : a+b, [w2v_model[j] for j in i])/len(i) for i in sparse_feat_seq_list])    
    else:
        raise NotImplementedError('input method %s out of method range'%method)
        
    for i in range(1, emb_dim+1):
        sparse_feat_seq_df.loc[:,'%s_%s'%(method,i)] = sparse_feat_seq_arr[:,i-1]
    
    sparse_feat_seq_df.drop(columns=[sparse_feat], inplace=True)
#     logger.info(sparse_feat_seq_df.shape)
    
    return sparse_feat_seq_df

In [19]:
w2v_model = KeyedVectors.load_word2vec_format(os.path.join(conf.DATA_DIR, 'creative_id_300_w2v.bin'), binary=True)

In [24]:
if '112276' in w2v_model:
    print('d')

d


In [6]:
train_preprocessed_df = pd.read_feather(os.path.join(conf.DATA_DIR, 'raw_train_round_one_df.feather'))

In [7]:
test_preprocessed_df = pd.read_feather(os.path.join(conf.DATA_DIR, 'raw_test_df.feather'))

In [8]:
train_preprocessed_df.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,gender,y
0,1,358946,216469,1,194447,8826.0,3,30464,NaN,6,1,16
1,1,652708,53492,1,49831,26858.0,3,29963,60.0,4,1,14
2,1,866186,220806,1,198186,NaN,12,18103,6.0,5,1,15
3,1,866186,63440,1,58787,87.0,2,22885,318.0,5,1,15
4,1,672438,20857,1,21792,NaN,18,22918,319.0,5,1,15


In [9]:
test_preprocessed_df.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,3892830,112276,1,101587,NaN,18,8371,54.0
1,1,3530749,14391,1,15775,1261.0,2,10988,6.0
2,1,3711528,208390,1,187259,1261.0,2,10925,6.0
3,1,3711528,236102,1,212289,1261.0,2,19056,98.0
4,1,3998628,90699,1,82895,NaN,18,10955,238.0


In [10]:
index_cols, cate_cols, cont_cols, label_cols = check_columns(train_preprocessed_df.dtypes.to_dict())
train_preprocessed_df.drop(columns=label_cols, inplace=True)
test_user_id = test_preprocessed_df['user_id'].unique()  # for further dividing data into train and test
preprocessed_df = pd.concat([train_preprocessed_df[index_cols + cate_cols + cont_cols], 
                             test_preprocessed_df[index_cols + cate_cols + cont_cols]],
                             axis=0)
del train_preprocessed_df, test_preprocessed_df
gc.collect()

7

In [11]:
preprocessed_df.head()

,time,user_id,creative_id,ad_id,product_id,advertiser_id,product_category,industry,click_times
0,1,358946,216469,194447,8826.0,30464,3,NaN,1
1,1,652708,53492,49831,26858.0,29963,3,60.0,1
2,1,866186,220806,198186,NaN,18103,12,6.0,1
3,1,866186,63440,58787,87.0,22885,2,318.0,1
4,1,672438,20857,21792,NaN,22918,18,319.0,1


In [12]:
sparse_feat_seq_df = _generate_emb_for_sparse_feat (
    preprocessed_df,
    'creative_id',
    'w2v',
    emb_dim=100,
    window=5,
    min_count=1,
    workers=20,
    sg=1
)

2020-05-15 10:26:13,922 - __main__ - INFO - _generate_emb_for_sparse_feat开始
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
2020-05-15 10:32:24,058 - __main__ - INFO - sparse_feat: creative_id , method: w2v, emd_dim: 100, window: 5, min_count: 1, workers: 20, sg: 1
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
2020-05-15 11:17:46,583 - __main__ - INFO - _generate_emb_for_sparse_feat已完成，共用时0:51:33


In [15]:
sparse_feat_seq_df.shape

(1900000, 101)

In [18]:
[10 if j in [1,2] else 1 for j in [1,2]  ]

[10, 10]

In [13]:
sub_dfs = dict(tuple(disk_smart_train_and_test_df.groupby(['model','serial_number'])))

In [33]:
def _apply_df(args):
    df, index_cols, label_cols, cont_cols, cate_cols = args
    return _create_daily_features(df, 
                                   index_cols, 
                                   label_cols, 
                                   cont_cols, 
                                   cate_cols)

def back_fill(sub_df, 
              back_fill_columns,
              freq='D',
              start_date=None, 
              end_date=None):
    """
    fill the missing value of a specific date with its nearest neighbour date,
    for rolling window
    :param sub_df:
    :param back_fill_columns: list of strings - filled with the nearest date data
    :param freq: str - frequency for filling missing date
    :param start_date: str - user-defined start date for sliding window
    :param end_date: str - user_defined end date for sliding window
    :return: df : pandas data-frame
    """
    back_fill_columns = [col for col in back_fill_columns if col in sub_df.columns]
    sub_df = sub_df.sort_values('dt')
    sub_df = sub_df.set_index('dt')
    start_date, end_date = sub_df.index[0] if start_date is None else start_date, \
                           sub_df.index[-1] if end_date is None else end_date
    date_range = pd.date_range(start_date, end_date, freq=freq)

    # back fill missing values
    sub_back_df = sub_df[back_fill_columns]
    sub_non_back_columns = list(set(sub_df.columns) - set(sub_back_df.columns))
    if sub_non_back_columns:
        sub_non_back_df = sub_df[sub_non_back_columns]
        sub_back_df = sub_back_df.reindex(date_range, method='pad')
        sub_non_back_df = sub_non_back_df.reindex(date_range, fill_value=0)
        df = pd.concat([sub_back_df, sub_non_back_df], axis=1).reset_index().rename(columns={'index': 'dt'})
        return df
    else:
        return sub_back_df.reindex(date_range, method='pad').reset_index().rename(columns={'index': 'dt'})

    
def _create_skew_kurt_cv(df, 
                          cols, 
                          window_list, 
                          window_size, 
                          min_periods,
                         ):
    """
    create statistics features about distribution
    :param df:
    :param cols:
    :param window_list:
    :param window_size:
    :param min_periods:
    :param start_index:
    :return:
    """
    dfs = []
    for i_window in window_list:
        target_df = df[cols].iloc[i_window * window_size:]
        df_skew = target_df.rolling(i_window * window_size, min_periods=min_periods).skew()
        df_skew.columns = [x + "_skew_" + str(i_window * window_size) for x in df_skew.columns]
        dfs.append(df_skew)
        df_kurt = target_df.rolling(i_window * window_size, min_periods=min_periods).kurt()
        df_kurt.columns = [x + "_kurt_" + str(i_window * window_size) for x in df_kurt.columns]
        dfs.append(df_kurt)
        df_cv = (target_df.rolling(i_window * window_size, min_periods=min_periods).std() /
                 target_df.rolling(i_window * window_size, min_periods=min_periods).mean())
        df_cv.columns = [x + "_cv_" + str(i_window * window_size) for x in df_cv.columns]
        dfs.append(df_cv)
    return dfs    

def _create_daily_features(df, 
                            index_cols,
                            label_cols,
                            cont_cols,
                            cate_cols,
                            window_list=[1, 2, 7],
                            window_size=7,
                            min_periods=1,
                            last_period_window=[1]):
    """
    create min, max, mean and std for different sliding window size
    :param df:
    :param window_list:
    :param window_size:
    :param min_periods:
    :return:
    """
    assert len(cont_cols)>0 and len(index_cols)>0 and len(label_cols)>0, \
    'cont_cols, index_cols and label_cols cannot be empty'
    
    origin_date = df['dt'].values
    back_fill_columns = index_cols + cont_cols
    df = back_fill(df, back_fill_columns=back_fill_columns, freq='D')
    df_index = df[index_cols + ['dt']]
    df_label = df[label_cols]
    
    if len(cate_cols):
        df_cate = df[cate_cols]
        dfs = [df_index, df_label, df_cate]
    else:
        dfs = [df_index, df_label]   
    df_sliding_cols = df[cont_cols]
    
    for i_window in window_list:
        target_df = df_sliding_cols
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).min()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_min_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).max()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_max_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).std()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_std_%s" % (i_window * window_size) for s in cont_cols]))))
        dfs.append((target_df.rolling(i_window * window_size, min_periods=min_periods).mean()
                    ).rename(
            columns=dict(zip(cont_cols, [s + "_mean_%s" % (i_window * window_size) for s in cont_cols]))))

    # TODO: the last period value for some columns and diff value 
    for last_period in last_period_window:
        if 0< last_period <= 7:
            for col in df_sliding_cols.columns:
                dfs.append((df_sliding_cols[[col]].shift(last_period)).rename(
                    columns=dict({col: '%s_diff_with_last_period_%s' % (col, last_period)})))

    # create skew, kurt and cv features
    dfs += _create_skew_kurt_cv(df_sliding_cols, 
                                 cont_cols, 
                                 [window_list[-1]],
                                 window_size=window_size,
                                 min_periods=min_periods,
                                 )
#   df = pd.concat(dfs, axis=1).fillna(DEFAULT_MISSING_FLOAT)
    df = pd.concat(dfs, axis=1)
    origin_date_df = df[df.date.isin(origin_date)]
    return origin_date_df

In [15]:
disk_smart_train_and_test_df.head()

,dt,flag,manufacturer,model,serial_number,smart_10_normalized,smart_12_normalized,smart_184_normalized,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_190_normalized,smart_191_normalized,smart_192_normalized,smart_193_normalized,smart_194_normalized,smart_195_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,tag
0,2018-04-06,0,A,1,disk_115552,100.0,100.0,100.0,100.0,100.0,99.0,71.0,100.0,100.0,100.0,29.0,47.0,100.0,100.0,200.0,80.0,96.0,100.0,100.0,94.0,60.0,0.0
1,2018-04-09,0,A,1,disk_115560,100.0,100.0,100.0,100.0,100.0,100.0,69.0,100.0,100.0,100.0,31.0,9.0,100.0,100.0,200.0,69.0,96.0,100.0,100.0,93.0,61.0,0.0
2,2018-04-04,0,A,1,disk_115561,100.0,100.0,100.0,100.0,100.0,98.0,69.0,100.0,100.0,100.0,31.0,64.0,100.0,100.0,200.0,77.0,96.0,100.0,100.0,93.0,61.0,0.0
3,2018-04-26,0,A,1,disk_115563,100.0,100.0,100.0,100.0,100.0,100.0,72.0,100.0,100.0,100.0,28.0,45.0,100.0,100.0,200.0,78.0,96.0,100.0,100.0,93.0,66.0,0.0
4,2018-04-17,0,A,1,disk_115563,100.0,100.0,100.0,100.0,100.0,100.0,73.0,100.0,100.0,100.0,27.0,47.0,100.0,100.0,200.0,82.0,96.0,100.0,100.0,93.0,67.0,0.0


In [16]:
_, cate_cols, cont_cols, label_cols = check_columns(disk_smart_train_and_test_df.dtypes.to_dict())

In [ ]:
index_cols = ['model', 'serial_number']
pool = multiprocessing.Pool(4)
result = pool.map_async(_apply_df, [(sub_dfs[key], index_cols, label_cols, cont_cols, cate_cols) \
                                     for key in sub_dfs.keys()])
pool.close()
fe_df = pd.concat(list(result.get()))

In [ ]:
mask = fe_df>='2018-04-01' 
mask &= fe_df <= '2018-07-31'

In [ ]:
train_fe_df = fe_df[mask]
test_fe_df = fe_df[~mask]

In [ ]:
train_fe_maj_label_df = train_fe_df[train_fe_df.tag==0] 
train_fe_maj_label_df = fe_df.sample(frac=0.6,random_state=1) 

In [6]:
fe_df = pd.read_feather(os.path.join(conf.DATA_DIR,'sample_by_clustering_0.35_fe_df_01_01_round_2_with_cluster_label.feather'))

In [7]:
fe_df.head()

,degradation_err_weight,degradation_err_weight_diff_for_last_period_7,degradation_err_weight_max_7,degradation_err_weight_min_7,degradation_err_weight_std_7,dt,err_weight,err_weight_diff_for_last_period_7,err_weight_max_7,err_weight_min_7,err_weight_std_7,flag,model,model_type,seek_err_weight,seek_err_weight_diff_for_last_period_7,seek_err_weight_max_7,seek_err_weight_min_7,seek_err_weight_std_7,serial_number,smart_12raw,smart_12raw_diff_for_last_period_7,smart_12raw_max_7,smart_12raw_min_7,smart_12raw_std_7,smart_184raw,smart_187_normalized,smart_188raw,smart_189_normalized,smart_189_normalized_diff_for_last_period_7,smart_189_normalized_max_7,smart_189_normalized_min_7,smart_189_normalized_std_7,smart_191_normalized,smart_191_normalized_diff_for_last_period_7,smart_191_normalized_max_7,smart_191_normalized_min_7,smart_191_normalized_std_7,smart_192raw,smart_192raw_diff_for_last_period_7,smart_192raw_max_7,smart_192raw_min_7,smart_192raw_std_7,smart_193_normalized,smart_193_normalized_diff_for_last_period_7,smart_193_normalized_max_7,smart_193_normalized_min_7,smart_193_normalized_std_7,smart_194raw,smart_194raw_diff_for_last_period_7,smart_194raw_max_7,smart_194raw_min_7,smart_194raw_std_7,smart_195_normalized,smart_195_normalized_diff_for_last_period_7,smart_195_normalized_max_7,smart_195_normalized_min_7,smart_195_normalized_std_7,smart_197raw,smart_198raw,smart_199raw,smart_199raw_diff_for_last_period_7,smart_199raw_max_7,smart_199raw_min_7,smart_199raw_std_7,smart_1_normalized,smart_1_normalized_diff_for_last_period_7,smart_1_normalized_max_7,smart_1_normalized_min_7,smart_1_normalized_std_7,smart_3_normalized,smart_3_normalized_diff_for_last_period_7,smart_3_normalized_max_7,smart_3_normalized_min_7,smart_3_normalized_std_7,smart_4raw,smart_4raw_diff_for_last_period_7,smart_4raw_max_7,smart_4raw_min_7,smart_4raw_std_7,smart_5raw,smart_5raw_diff_for_last_period_7,smart_5raw_max_7,smart_5raw_min_7,smart_5raw_slope_for_last_duration_7,smart_5raw_std_7,smart_7_normalized,smart_7_normalized_diff_for_last_period_7,smart_7_normalized_max_7,smart_7_normalized_min_7,smart_7_normalized_std_7,smart_9_normalized,tag
0,34.686664,NaN,34.686664,34.686664,NaN,2017-07-09,3.746948,NaN,3.746948,3.746948,NaN,0,1,0,36.639879,NaN,36.639879,36.639879,NaN,disk_89624,20.0,NaN,20.0,20.0,NaN,0.0,0.0,0.0,2.0,NaN,2.0,2.0,NaN,0.0,NaN,0.0,0.0,NaN,19.0,NaN,19.0,19.0,NaN,0.0,NaN,0.0,0.0,NaN,28.0,NaN,28.0,28.0,NaN,52.0,NaN,52.0,52.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,81.0,NaN,81.0,81.0,NaN,96.0,NaN,96.0,96.0,NaN,20.0,NaN,20.0,20.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,85.0,NaN,85.0,85.0,NaN,8.0,1
1,35.366899,NaN,35.366899,35.366899,NaN,2017-07-09,11.309656,NaN,11.309656,11.309656,NaN,0,1,0,39.787716,NaN,39.787716,39.787716,NaN,disk_143171,12.0,NaN,12.0,12.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,11.0,NaN,11.0,11.0,NaN,0.0,NaN,0.0,0.0,NaN,29.0,NaN,29.0,29.0,NaN,54.0,NaN,54.0,54.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,80.0,NaN,80.0,80.0,NaN,95.0,NaN,95.0,95.0,NaN,12.0,NaN,12.0,12.0,NaN,27.0,NaN,27.0,27.0,NaN,NaN,93.0,NaN,93.0,93.0,NaN,20.0,1
2,40.707018,NaN,40.707018,40.707018,NaN,2017-07-09,3.436182,NaN,3.436182,3.436182,NaN,0,1,0,40.003033,NaN,40.003033,40.003033,NaN,disk_28506,19.0,NaN,19.0,19.0,NaN,0.0,0.0,0.0,6.0,NaN,6.0,6.0,NaN,0.0,NaN,0.0,0.0,NaN,17.0,NaN,17.0,17.0,NaN,0.0,NaN,0.0,0.0,NaN,31.0,NaN,31.0,31.0,NaN,45.0,NaN,45.0,45.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,81.0,NaN,81.0,81.0,NaN,96.0,NaN,96.0,96.0,NaN,21.0,NaN,21.0,21.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,92.0,NaN,92.0,92.0,NaN,29.0,1
3,31.103376,NaN,31.103376,31.103376,NaN,2017-07-09,16.168701,NaN,16.168701,16.168701,NaN,0,2,1,36.974979,NaN,36.974979,36.974979,NaN,disk_118621,7.0,NaN,7.0,7.0,NaN,0.0,0.0,1.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,13.0,NaN,13.0,13.0,NaN,0.0,NaN,0.0,0.0,NaN,29.0,NaN,29.0,29.0,NaN,37.0,NaN,37.0,37.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,68.0,NaN,68.0,68.0,NaN,87.0,NaN,87.0,87.0,NaN,7.0,NaN,7.0,7.0,NaN,48.0,NaN,48.0,48.0,NaN,NaN,86.0,NaN,86.0,86.0,NaN,11.0,1
4,33.063739,NaN,33.0

In [9]:
len(fe_df.columns)

93

In [11]:
[col for col in fe_df.columns if 'normalized' in col.split('_')]

['smart_187_normalized',
 'smart_189_normalized',
 'smart_189_normalized_diff_for_last_period_7',
 'smart_189_normalized_max_7',
 'smart_189_normalized_min_7',
 'smart_189_normalized_std_7',
 'smart_191_normalized',
 'smart_191_normalized_diff_for_last_period_7',
 'smart_191_normalized_max_7',
 'smart_191_normalized_min_7',
 'smart_191_normalized_std_7',
 'smart_193_normalized',
 'smart_193_normalized_diff_for_last_period_7',
 'smart_193_normalized_max_7',
 'smart_193_normalized_min_7',
 'smart_193_normalized_std_7',
 'smart_195_normalized',
 'smart_195_normalized_diff_for_last_period_7',
 'smart_195_normalized_max_7',
 'smart_195_normalized_min_7',
 'smart_195_normalized_std_7',
 'smart_1_normalized',
 'smart_1_normalized_diff_for_last_period_7',
 'smart_1_normalized_max_7',
 'smart_1_normalized_min_7',
 'smart_1_normalized_std_7',
 'smart_3_normalized',
 'smart_3_normalized_diff_for_last_period_7',
 'smart_3_normalized_max_7',
 'smart_3_normalized_min_7',
 'smart_3_normalized_std_7',

In [13]:
[col for col in fe_df.columns if 'normalized' not in col.split('_')]

['degradation_err_weight',
 'degradation_err_weight_diff_for_last_period_7',
 'degradation_err_weight_max_7',
 'degradation_err_weight_min_7',
 'degradation_err_weight_std_7',
 'dt',
 'err_weight',
 'err_weight_diff_for_last_period_7',
 'err_weight_max_7',
 'err_weight_min_7',
 'err_weight_std_7',
 'flag',
 'model',
 'model_type',
 'seek_err_weight',
 'seek_err_weight_diff_for_last_period_7',
 'seek_err_weight_max_7',
 'seek_err_weight_min_7',
 'seek_err_weight_std_7',
 'serial_number',
 'smart_12raw',
 'smart_12raw_diff_for_last_period_7',
 'smart_12raw_max_7',
 'smart_12raw_min_7',
 'smart_12raw_std_7',
 'smart_184raw',
 'smart_188raw',
 'smart_192raw',
 'smart_192raw_diff_for_last_period_7',
 'smart_192raw_max_7',
 'smart_192raw_min_7',
 'smart_192raw_std_7',
 'smart_194raw',
 'smart_194raw_diff_for_last_period_7',
 'smart_194raw_max_7',
 'smart_194raw_min_7',
 'smart_194raw_std_7',
 'smart_197raw',
 'smart_198raw',
 'smart_199raw',
 'smart_199raw_diff_for_last_period_7',
 'smart_19